In [30]:
from bs4 import BeautifulSoup
import re
import requests
from time import sleep
from nltk.corpus import stopwords
import pandas as pd
import os
import numpy as np
from selenium import webdriver

In [23]:
def scrapying_selenium(url):
    headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36"}   
    browser = webdriver.Chrome("/Applications/chromedriver") 
    browser.set_page_load_timeout(5)
    browser.get(url) #Connects to Website
    soup = BeautifulSoup(browser.page_source)# Get the html from the site
    browser.quit()
    return soup

In [12]:
def cleaning_site(website):
    headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36"}   
    browser = webdriver.Chrome("/Applications/chromedriver") 
    browser.set_page_load_timeout(10)
    browser.get(website) #Connects to Website

    soup = BeautifulSoup(browser.page_source)# Get the html from the site
    browser.quit()
    job_descriptions = soup.findAll('div', class_="rich-text", itemprop="description")
    
    descriptions = [job.text.strip().encode('ascii','ignore') for job in job_descriptions]
    return descriptions

In [13]:
def init_mongo_client():
    # Initiate Mongo client
    client = pymongo.MongoClient()

    # Access database created for these articles
    db = client.job_descriptions
    
    # Access collection created for these articles
    job_descriptions = db.zip_jobs

    # Access articles collection in db and return collection pointer.
    return db.zip_jobs


In [14]:
website= 'https://www.linkedin.com/jobs2/view/94823687?refId=7396fdb5-579a-42e3-8b8a-23f311d5c4c1&trk=jobs_home_click_jymbii'

In [12]:
data = cleaning_site(website)

In [15]:
def get_url(job, city, state):

    query_job = job.split()
    query_job = "+".join(query_job)

    query_city = city.split()
    query_city = "+".join(query_city)

    url = 'https://www.linkedin.com/jobs/search?keywords=' + query_job + '&location=' + query_city + '%2C+' + state

    return url


In [16]:
get_url('Data Scientist', 'San Francisco', 'California')

'https://www.linkedin.com/jobs/search?keywords=Data+Scientist&location=San+Francisco%2C+California'

In [25]:
def scraping_linkedin(job, country = 'us'):

    if job is None:
        print 'Please insert a job'
        return


    query_job = job.split()
    query_job = "+".join(query_job)
    zip_url = 'https://www.linkedin.com/jobs/search?keywords=' + query_job + '&locationId=' + country + ':0&start=0'
    '''   
        https://www.linkedin.com/jobs/search?keywords=Data+Scientist&locationId=us:0&start=0
        https://www.linkedin.com/jobs/search?keywords=Data+Scientist&locationId=es:0&start=0
        https://www.linkedin.com/jobs/search?keywords=Data+Scientist&locationId=es:0&start=50&count=25&trk=jobs_jserp_pagination_3
    '''
    soup = scrapying_selenium(zip_url)
    # Scraping number of pages to loop through

    job_numbers = soup.find('div', class_="results-context").text
    job_numbers = job_numbers.split(' ')
    
    # When the total is larger than 1000 the comma in the string cannot be transformed to int
    try:
        job_numbers = "".join(job_numbers[0].split(','))
    except:
        pass    

    jobs_total = int(job_numbers)
    total_pages = jobs_total/25
    final_descriptions = [] 
    final_companies    = []
    final_titles       = []
    final_locations    = []

    #Loop that iterates through every page
    for page in xrange(total_pages):

        #defining the url for every page  
        url = 'https://www.linkedin.com/jobs/search?keywords=' + query_job + '&locationId=' + country + ':0&start=' + str(page*25)

        print 'Scraping page %d of %d pages' %(page, total_pages)
        
        soup = scrapying_selenium(zip_url)
        
        #Scraping State and cleaning data
        locations = soup.findAll('span', itemprop="addressLocality")
        locations = [location.text.strip().encode('ascii','ignore') for location in locations]        
    
        #Scraping company and cleaning data       
        companies = soup.findAll('span', class_="company-name-text")
        companies = [company.text.strip().encode('ascii','ignore') for company in companies]        
    
        #Scraping job and cleaning data 
        titles = soup.findAll('span', class_="job-title-text")
        titles = [title.text.strip().encode('ascii','ignore') for title in titles]        

        # Extracting every job description link of the current page in indeed
        links = soup.findAll('a', class_="job-title-link")
        links = [line.get('href') for line in links]
        job_links = [link for link in links if 'jobs2' in link]

        #Scraping and cleaning data from the job descriptions links
        job_descriptions =  []
        for url in job_links:
            job_descriptions.append(cleaning_site(url))

 
    
        final_descriptions.extend(job_descriptions)
        final_companies.extend(companies)
        final_locations.extend(locations)
        final_titles.extend(titles)
        sleep(5)
        if page == 3:
            break  
    data = pd.DataFrame({'job_title' : final_titles, 'job_description' : final_descriptions, 'company' : final_companies, 'location' : final_locations})
    return data

In [26]:
data = scraping_linkedin('Data Scientist')

Scraping page 0 of 1566 pages
Scraping page 1 of 1566 pages
Scraping page 2 of 1566 pages
Scraping page 3 of 1566 pages


In [29]:
for i in data.company:
    print i
    print '**********'

Code and Theory
**********
Intel Corporation
**********
Intel Corporation
**********
Moven
**********
CoreLogic
**********
Redpoint Ventures
**********
HireStrong
**********
LiftPoint Consulting (formerly M Squared Group)
**********
National Security Agency
**********
Wolters Kluwer Health
**********
Auction.com
**********
Intel Corporation
**********
Lotame
**********
General Motors
**********
Conclusive Analytics
**********
Zynga
**********
Nielsen
**********
Booz Allen Hamilton
**********
Electronic Arts (EA)
**********
HomeAway.com
**********
Texas Instruments
**********
Expedia, Inc.
**********
STATS LLC
**********
Alpine Data Labs
**********
Blastworks Inc.
**********
Code and Theory
**********
Intel Corporation
**********
Intel Corporation
**********
Moven
**********
CoreLogic
**********
Redpoint Ventures
**********
HireStrong
**********
LiftPoint Consulting (formerly M Squared Group)
**********
National Security Agency
**********
Wolters Kluwer Health
**********
Auction.com
***